<a href="https://colab.research.google.com/github/JacobHallWeather/JacobHallWeather/blob/main/Verif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title Default title text
import requests
import json
import os
from datetime import datetime
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define offices and their associated public zones
office_zones = {
    'PQR': [
        'WAZ201', 'WAZ202', 'WAZ203', 'WAZ204', 'WAZ205', 'WAZ206', 'WAZ207', 'WAZ208', 'WAZ209', 'WAZ210', 'WAZ211',
        'ORZ101', 'ORZ102', 'ORZ103', 'ORZ104', 'ORZ105', 'ORZ106', 'ORZ107', 'ORZ108', 'ORZ109', 'ORZ110', 'ORZ111',
        'ORZ112', 'ORZ113', 'ORZ114', 'ORZ115', 'ORZ116', 'ORZ117', 'ORZ118', 'ORZ119', 'ORZ120', 'ORZ121', 'ORZ122',
        'ORZ123', 'ORZ124', 'ORZ125', 'ORZ126', 'ORZ127', 'ORZ128'
    ],
    'SEW': [
        'PZZ110', 'PZZ130', 'PZZ131', 'PZZ132', 'PZZ133', 'PZZ134', 'PZZ135', 'PZZ150', 'PZZ153', 'PZZ156', 'PZZ170',
        'PZZ173', 'PZZ176', 'WAZ001', 'WAZ503', 'WAZ504', 'WAZ506', 'WAZ507', 'WAZ509', 'WAZ510', 'WAZ511', 'WAZ512',
        'WAZ513', 'WAZ514', 'WAZ515', 'WAZ516', 'WAZ517', 'WAZ555', 'WAZ556', 'WAZ558', 'WAZ559', 'WAZ567', 'WAZ568',
        'WAZ569'
    ],
    'MFR': [
        'CAZ080', 'CAZ081', 'CAZ082', 'CAZ083', 'CAZ084', 'CAZ085', 'ORZ021', 'ORZ022', 'ORZ023', 'ORZ024', 'ORZ025',
        'ORZ026', 'ORZ027', 'ORZ028', 'ORZ029', 'ORZ030', 'ORZ031', 'PZZ350', 'PZZ356', 'PZZ370', 'PZZ376'
    ]
}

# Function to delete existing JSON files, including synoptic data files
def delete_existing_files():
    files_to_delete = ['hazards_data.json', 'lsr_data.json']
    # Delete hazard and LSR data files
    for file in files_to_delete:
        if os.path.exists(file):
            os.remove(file)

    # Delete all synoptic data files
    for file in os.listdir():
        if file.startswith('synoptic_data_') and file.endswith('.json'):
            os.remove(file)

# Convert date to ISO 8601 format with time set to 00:00:00Z
def convert_to_iso8601(date_obj):
    return date_obj.strftime('%Y-%m-%dT00:00:00Z')

# Function to extract the zone ID and remove 'Z'
def extract_zone_id(zone_url):
    """Extract the zone ID from a full URL and remove the 'Z' character."""
    zone_id = zone_url.split('/')[-1]  # Extract the last part of the URL
    return zone_id.replace('Z', '')  # Remove the 'Z' character

# Function to make API calls based on user input
def make_api_calls(office, start_date_iso, end_date_iso):
    delete_existing_files()  # Delete existing files before running the API calls

    # Remove 'Z' from each zone name
    zones = office_zones[office]

    # 1. API call to retrieve hazard alerts (using ISO format)
    hazards_url = f"https://api.weather.gov/alerts?start={start_date_iso}&end={end_date_iso}&status=actual&message_type=alert,update,cancel&zone={','.join(zones)}"
    hazards_response = requests.get(hazards_url)
    hazards_data = hazards_response.json()

    # Save the hazards data to a JSON file
    with open('hazards_data.json', 'w') as f:
        json.dump(hazards_data, f, indent=4)

    return hazards_data

# Function to fetch and return synoptic data for a specific hazard and zone
def fetch_synoptic_data_for_zone(hazard_id, zone_url, start_date, end_date):
    """Fetch synoptic data using the correct zone ID."""
    synoptic_api_key = "a1f0e14ba19c420fbdea8e38bde4a22d"  # Replace with your actual Synoptic API key
    zone_id = extract_zone_id(zone_url)  # Extract and modify the zone ID
    start_date_synoptic = start_date.strftime('%Y%m%d%H%M')
    end_date_synoptic = end_date.strftime('%Y%m%d%H%M')

    synoptic_url = f"https://api.synopticdata.com/v2/stations/timeseries?nwszone={zone_id}&start={start_date_synoptic}&end={end_date_synoptic}&token={synoptic_api_key}"
    synoptic_response = requests.get(synoptic_url)
    synoptic_data = synoptic_response.json()

    return synoptic_data

# User input widgets
office_input = widgets.Dropdown(
    options=office_zones.keys(),
    value='PQR',
    description='Office:',
    disabled=False
)

start_date_input = widgets.DatePicker(
    description='Start Date',
    disabled=False
)

end_date_input = widgets.DatePicker(
    description='End Date',
    disabled=False
)

submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='info',
    tooltip='Click to submit',
    icon='check'
)

# Dropdowns for alerts, zones, stations, and variables
alert_dropdown = widgets.Dropdown(description='Alert:')
zone_dropdown = widgets.Dropdown(description='Zone:')
station_dropdown = widgets.Dropdown(description='Station:')
variable_dropdown = widgets.Dropdown(description='Variable:', options=['Temperature', 'Wind Speed', 'Precipitation'])

# Display the widgets
display(office_input, start_date_input, end_date_input, submit_button, alert_dropdown, zone_dropdown, station_dropdown, variable_dropdown)

# Update station dropdown based on the selected zone
def update_station_dropdown(*args):
    selected_zone = zone_dropdown.value
    if selected_zone:
        # Fetch synoptic data to get available stations for the selected zone
        selected_alert = alert_dropdown.value
        for alert in loaded_hazards['features']:
            event_name = alert['properties']['event']
            vtec_code = alert['properties']['parameters']['VTEC'][0] if 'VTEC' in alert['properties']['parameters'] else 'N/A'
            alert_name = f"{event_name}_{vtec_code}"
            if alert_name == selected_alert:
                onset = alert['properties'].get('onset')
                ends = alert['properties'].get('ends')
                start_date = datetime.strptime(onset, "%Y-%m-%dT%H:%M:%S%z")
                end_date = datetime.strptime(ends, "%Y-%m-%dT%H:%M:%S%z")
                synoptic_data = fetch_synoptic_data_for_zone(selected_alert, selected_zone, start_date, end_date)

                if 'STATION' in synoptic_data:
                    station_names = [station['NAME'] for station in synoptic_data['STATION']]
                    station_dropdown.options = station_names

zone_dropdown.observe(update_station_dropdown, names='value')

# Update zone dropdown options based on the selected alert
def update_zone_dropdown(*args):
    selected_alert = alert_dropdown.value
    if selected_alert:
        for alert in loaded_hazards['features']:
            event_name = alert['properties']['event']
            vtec_code = alert['properties']['parameters']['VTEC'][0] if 'VTEC' in alert['properties']['parameters'] else 'N/A'
            alert_name = f"{event_name}_{vtec_code}"
            if alert_name == selected_alert:
                zone_options = alert['properties']['affectedZones']
                zone_dropdown.options = zone_options

alert_dropdown.observe(update_zone_dropdown, names='value')

# Define what happens when the submit button is clicked
def on_button_clicked(b):
    clear_output(wait=True)
    display(office_input, start_date_input, end_date_input, submit_button, alert_dropdown, zone_dropdown, station_dropdown, variable_dropdown)

    office = office_input.value
    start_date_iso = convert_to_iso8601(start_date_input.value)
    end_date_iso = convert_to_iso8601(end_date_input.value)

    global loaded_hazards
    loaded_hazards = make_api_calls(office, start_date_iso, end_date_iso)

    # Filter hazards that have valid onset, ends, and VTEC
    valid_hazards = [
        alert for alert in loaded_hazards['features']
        if alert['properties'].get('onset') and alert['properties'].get('ends') and 'VTEC' in alert['properties']['parameters']
    ]

    # Update alert dropdown with hazard name and VTEC
    alert_dropdown.options = [
        f"{alert['properties']['event']}_{alert['properties']['parameters']['VTEC'][0]}"
        for alert in valid_hazards
    ]

submit_button.on_click(on_button_clicked)

# Define function to display data in a table for the selected station
def display_data_table(*args):
    selected_alert = alert_dropdown.value
    selected_zone = zone_dropdown.value
    selected_station = station_dropdown.value
    selected_variable = variable_dropdown.value

    if selected_alert and selected_zone and selected_station:
        for alert in loaded_hazards['features']:
            event_name = alert['properties']['event']
            vtec_code = alert['properties']['parameters']['VTEC'][0] if 'VTEC' in alert['properties']['parameters'] else 'N/A'
            alert_name = f"{event_name}_{vtec_code}"
            if alert_name == selected_alert:
                onset = alert['properties'].get('onset')
                ends = alert['properties'].get('ends')

                # Check if onset and ends are valid strings
                if onset and ends:
                    start_date = datetime.strptime(onset, "%Y-%m-%dT%H:%M:%S%z")
                    end_date = datetime.strptime(ends, "%Y-%m-%dT%H:%M:%S%z")
                    synoptic_data = fetch_synoptic_data_for_zone(selected_alert, selected_zone, start_date, end_date)

                    # Check if the 'STATION' key exists
                    if 'STATION' in synoptic_data:
                        # Filter data for the selected station
                        for station in synoptic_data['STATION']:
                            if station['NAME'] == selected_station:
                                df = pd.DataFrame(station['OBSERVATIONS'])
                                if selected_variable.lower() in df.columns:
                                    filtered_df = df[['DATE', selected_variable.lower()]]
                                else:
                                    filtered_df = df

                                display(filtered_df)

# Observe changes in dropdowns and update table
alert_dropdown.observe(display_data_table, names='value')
zone_dropdown.observe(display_data_table, names='value')
station_dropdown.observe(display_data_table, names='value')
variable_dropdown.observe(display_data_table, names='value')


Dropdown(description='Office:', options=('PQR', 'SEW', 'MFR'), value='PQR')

DatePicker(value=None, description='Start Date')

DatePicker(value=None, description='End Date')

Button(button_style='info', description='Submit', icon='check', style=ButtonStyle(), tooltip='Click to submit'…

Dropdown(description='Alert:', options=(), value=None)

Dropdown(description='Zone:', options=(), value=None)

Dropdown(description='Station:', options=(), value=None)

Dropdown(description='Variable:', options=('Temperature', 'Wind Speed', 'Precipitation'), value='Temperature')